In [9]:
import numpy as np


def f(x):
    """Right-hand side of the ODE."""
    return (x**2 + 2*x + 2)/(x + 1)


def solve_bvp(N, gamma1=1.38294):
    """
    Solve:
        u'' + (x+1) u' - u = (x^2 + 2x + 2)/(x+1),
    with boundary conditions u(0)=0, u(1)=gamma1,
    using Budeev–Timukhin (variant 4) finite differences.
    """
    h = 1.0 / N
    x = np.linspace(0, 1, N+1)  # x_0=0,...,x_N=1

    # We will build an (N-1)x(N-1) system for u_1..u_{N-1}.
    A = np.zeros((N-1, N-1))
    F = np.zeros(N-1)

    for i in range(1, N):
        xi = x[i]
        # r_i = (p_i / 2) * h = ((x_i+1)/2) * h
        r_i = (xi + 1) * h / 2.0

        # alpha_i = 1 + |r_i|^3 / (1 + |r_i| + r_i^2)
        alpha_i = 1.0 + abs(r_i)**3 / (1.0 + abs(r_i) + r_i**2)

        a_i = (alpha_i - r_i) / (h**2)
        c_i = (alpha_i + r_i) / (h**2)
        # q(x_i) = -1 => b_i = -2 alpha_i/h^2 + q_i
        b_i = -2.0*alpha_i/(h**2) - 1.0

        # Fill row (i-1) in matrix A:
        if i-1 > 0:
            A[i-1, i-2] = a_i  # sub-diagonal
        A[i-1, i-1] = b_i     # main diagonal
        if i-1 < (N-2):
            A[i-1, i] = c_i   # super-diagonal

        # Right-hand side:
        F[i-1] = f(xi)

    # Enforce boundary conditions:
    #  - For i=1 row, subtract a_1*u0 => but u0=0 => no effect.
    #  - For i=N-1 row, subtract c_{N-1}*uN from RHS.
    i = N-1
    xi = x[i]
    r_i = (xi + 1)*h/2.0
    alpha_i = 1.0 + abs(r_i)**3 / (1.0 + abs(r_i) + r_i**2)
    c_i = (alpha_i + r_i)/(h**2)
    F[N-2] -= c_i * gamma1

    # Solve for u_1..u_{N-1}:
    u_inner = np.linalg.solve(A, F)

    # Construct full solution array:
    U = np.zeros(N+1)
    U[0] = 0.0
    U[1:N] = u_inner
    U[N] = gamma1

    return x, U

In [10]:
N = 10
x, U_approx = solve_bvp(N)
U_exact = (x+1)*np.log(x+1)
error = U_approx - U_exact

print(" i    x_i        U_approx        U_exact         error")
print("----------------------------------------------------------")
for i in range(N+1):
    print(
        f"{i:2d}  {x[i]:8.5f}  {U_approx[i]:14.8f}  {U_exact[i]:14.8f}  {error[i]:14.8e}")
    

 i    x_i        U_approx        U_exact         error
----------------------------------------------------------
 0   0.00000      0.00000000      0.00000000  0.00000000e+00
 1   0.10000      0.10433977      0.10484120  -5.01426812e-04
 2   0.20000      0.21783167      0.21878587  -9.54200315e-04
 3   0.30000      0.33971122      0.34107354  -1.36231968e-03
 4   0.40000      0.46933077      0.47106113  -1.73036058e-03
 5   0.50000      0.60613456      0.60819766  -2.06310581e-03
 6   0.60000      0.74964051      0.75200581  -2.36529932e-03
 7   0.70000      0.89942654      0.90206803  -2.64148191e-03
 8   0.80000      1.05512011      1.05801600  -2.89588418e-03
 9   0.90000      1.21639002      1.21952238  -3.13236159e-03
10   1.00000      1.38294000      1.38629436  -3.35436112e-03
